# Retrieval Sweep Results Analysis
이 노트북은 `scripts/sweep_retrieval.py`가 생성한 CSV 결과를 로드해
구성별 성능을 비교하고 상위 조합을 선별합니다.


In [ ]:

import pandas as pd
from pathlib import Path
from caas_jupyter_tools import display_dataframe_to_user

results_dir = Path('/mnt/data/results')
summary_csv = list(results_dir.glob('*.csv'))
summary_csv


In [ ]:

import pandas as pd
from pathlib import Path

# 파일 경로를 직접 지정하세요
summary_path = Path('/mnt/data/results/sweep_run.csv')
per_query_path = Path('/mnt/data/results/sweep_run_per_query.csv')

summary = pd.read_csv(summary_path)
per_query = pd.read_csv(per_query_path)

summary.head(), per_query.head()


## 상위 조합(top-N) 선정

In [ ]:

# 간단한 스코어: hit_rate 0.6, kw_hit_rate 0.3, - avg_latency_s 0.1 가중합
summary = summary.copy()
summary['score'] = 0.6*summary['hit_rate'] + 0.3*summary['kw_hit_rate'] - 0.1*summary['avg_latency_s']
top = summary.sort_values('score', ascending=False).head(10)
top


## 시각화 (matplotlib, 색상 미지정 규칙 준수)

In [ ]:

import matplotlib.pyplot as plt

plt.figure()
plt.scatter(summary['avg_latency_s'], summary['hit_rate'])
plt.xlabel('Avg Latency (s)')
plt.ylabel('Hit Rate')
plt.title('Latency vs Hit Rate')
plt.show()


## 의도/쿼리 유형별 성능 (선택: per_query CSV에 라벨 칼럼 추가 시)

In [ ]:

# 예시: per_query['intent']가 있다면 그룹별 평균을 볼 수 있습니다.
if 'intent' in per_query.columns:
    print(per_query.groupby('intent')[['hit','kw_hit','coverage','latency_s']].mean())
else:
    print('per_query.csv에 intent 칼럼이 없네요. 원한다면 eval_set에 intent를 넣고 스크립트에서 내려주세요.')
